In [ ]:
import sys
import os
from ml4cvd.arguments import parse_args
from ml4cvd.models import make_multimodal_multitask_model, saliency_map
from ml4cvd.tensor_generators import test_train_valid_tensor_generators, big_batch_from_minibatch_generator
from ml4cvd.recipes import run
import h5py
import numpy as np
import matplotlib
# font = {'family' : 'Arial',
#         'size'   : 30}
# matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
from typing import List, Dict, Callable
from collections import defaultdict
import pandas as pd

from ml4cvd.arguments import _get_tmap
from ml4cvd.TensorMap import TensorMap
from ml4cvd.tensor_from_file import TMAPS

import numpy as np
USER = 'pdiachil'
HOME_PATH = '/home/' + USER

In [ ]:
sys.argv = ['train',
            '--mode', 'train',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'/mnt/disks/ecg-rest-37k-tensors/2019-11-04/',
            '--input_tensors', 'ecg_rest',
            '--output_tensors', 'qrs-duration', 'p-duration', 'qt-interval', 'pq-interval', 'rr-interval',
            '--id', 'ecg_rest_pretraining_intervals',
            '--inspect_model',
            '--training_steps', '810',
            '--validation_steps', '230',
            '--test_steps', '115',
            '--batch_size', '32',
            '--epochs', '3',
            '--patience', '12'            
           ]
args = parse_args()

In [ ]:
run(args)

In [ ]:
sys.argv = ['infer',
            '--mode', 'infer',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'/mnt/disks/ecg-rest-37k-tensors/2019-11-04/',
            '--input_tensors', 'ecg_rest',
            '--output_tensors', 'qrs-duration', 'p-duration', 'qt-interval', 'pq-interval', 'rr-interval',
            '--id', 'ecg_rest_pretraining_intervals',
            '--model_file', f'{HOME_PATH}/ukbb_output/ecg_rest_pretraining_intervals/ecg_rest_pretraining_intervals.hd5',
            '--training_steps', '810',
            '--validation_steps', '230',
            '--test_steps', '115',
            '--batch_size', '32',
            '--epochs', '1',
            '--patience', '12'            
           ]
args = parse_args()

In [ ]:
import shutil
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(keep_paths=True, **args.__dict__)
model = make_multimodal_multitask_model(**args.__dict__)
for gen in [generate_train, generate_valid, generate_test]:
    data_len = gen.true_epoch_lens[0]
    cnt = 0
    for (data, labels, paths) in gen: 
        if cnt >= data_len: break
        in_tensor = data[args.tensor_maps_in[0].input_name()]
        for tm in args.tensor_maps_out:
            for channel in tm.channel_map:
                gradients = saliency_map(in_tensor, model, tm.output_name(), tm.channel_map[channel])
                for i, path in enumerate(paths):
                    fname = os.path.basename(path)
                    if not os.path.isfile(os.path.join('/mnt/disks/ecg-rest-37k-tensors-saliency/2020-02-03/', fname)):
                        shutil.copy(path, os.path.join('/mnt/disks/ecg-rest-37k-tensors-saliency/2020-02-03/', fname))
                    with h5py.File(os.path.join(args.output_folder, fname)) as hd5:
                        try:
                            hd5.create_dataset(f'{args.tensor_maps_in[0].group}/{tm.group}/{tm.input_name()}_saliency',
                                               data = gradients[i, :, :])            
                        except RuntimeError as e:
                            print(f'{args.tensor_maps_in[0].group}/{tm.group}/{tm.input_name()}_saliency already exists in {fname}')        
        cnt += len(paths)

In [ ]:
def plot_saliency(hd5, in_name, out_name):
    in_tmap = _get_tmap(in_name)
    out_tmap = _get_tmap(out_name)
    in_tensor = in_tmap.tensor_from_file(in_tmap, hd5)
    print()
    out_tensor = np.array(hd5[f'{in_tmap.group}/{out_tmap.group}/{out_tmap.input_name()}_saliency'])
    f, ax = plt.subplots()
    f.set_size_inches(16, 9)
    ax.plot(in_tensor[:, 0])
    ax.plot(np.abs(out_tensor[:, 0]))

In [ ]:
%matplotlib inline
with h5py.File('/mnt/disks/ecg-rest-37k-tensors-saliency/2020-02-03/1000503.hd5') as hd5:
    plot_saliency(hd5, 'ecg_rest', 'pq-interval')

# Learn saliency maps

In [ ]:
sys.argv = ['train',
            '--mode', 'train',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'/mnt/disks/ecg-rest-37k-tensors-saliency/2020-02-03/',
            '--input_tensors', 'ecg_rest',
            '--output_tensors', 'ecg_rest_saliency',
            '--id', 'ecg_rest_pretraining_intervals',
            '--model_file', f'{HOME_PATH}/ukbb_output/ecg_rest_pretraining_intervals/ecg_rest_pretraining_intervals.hd5',
            '--training_steps', '810',
            '--validation_steps', '230',
            '--test_steps', '115',
            '--batch_size', '32',
            '--epochs', '1',
            '--patience', '12'            
           ]
args = parse_args()

# Learn under the mask

In [ ]:
sys.argv = ['train',
            '--mode', 'train',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'/mnt/disks/ecg-rest-37k-tensors/2019-11-04/',
            '--input_tensors', 'ecg_rest_masked',
            '--output_tensors', 'ecg_rest_output',
            '--u_connect',
            '--inspect_model',
            '--id', 'ecg_rest_pretraining_mask',
            '--training_steps', '810',
            '--validation_steps', '230',
            '--test_steps', '115',
            '--batch_size', '32',
            '--epochs', '1',
            '--patience', '12'            
           ]
args = parse_args()

In [ ]:
run(args)

In [ ]:
sys.argv = ['train',
            '--mode', 'train',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'/mnt/disks/ecg-rest-37k-tensors/2019-11-04/',
            '--input_tensors', 'ecg_rest_masked',
            '--output_tensors', 'ecg_rest_output',
            '--model_file', f'/home/pdiachil/ukbb_output/ecg_rest_pretraining_mask/ecg_rest_pretraining_mask.hd5',
            '--u_connect',
            '--id', 'ecg_rest_pretraining_mask',
            '--training_steps', '810',
            '--validation_steps', '230',
            '--test_steps', '115',
            '--batch_size', '32',
            '--epochs', '10',
            '--patience', '12'            
           ]
args = parse_args()

In [ ]:
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(keep_paths=True, **args.__dict__)
model = make_multimodal_multitask_model(**args.__dict__)

In [ ]:
%matplotlib inline
for gen in [generate_train, generate_valid, generate_test]:
    cnt = 0
    for (data, labels, paths) in gen: 
        in_tensor = data[args.tensor_maps_in[0].input_name()]
        label_tensor = labels[args.tensor_maps_out[0].output_name()]
        f, ax = plt.subplots(3, 1)
        ax[0].plot(in_tensor[0, :, 0])
        ax[1].plot(np.argmax(label_tensor[0, :, 0, :], axis=1))
        out_tensor = model.predict(in_tensor)
        ax[2].plot(np.argmax(out_tensor[0, :, 0, :]))
        break
    break

In [ ]:
label_tensor[0, :, 0, :].shape

In [ ]:
for layer in model.layers[:-4]:
    layer.trainable = False
    print(layer)
model.save(f'{HOME_PATH}/ukbb_output/ecg_rest_pretraining_mask_transfer/ecg_rest_pretraining_mask_transfer.hd5')

In [ ]:
sys.argv = ['train',
            '--mode', 'train',
            '--output_folder', f'{HOME_PATH}/ukbb_output/',
            '--tensors', f'{HOME_PATH}/ludb_tensors/',
            '--input_tensors', 'ecg_rest_ludb',
            '--output_tensors', 'ecg_rest_ludb_segmentation_coarse',
            '--id', 'ecg_rest_pretraining_mask_transfer',
            '--model_file', f'{HOME_PATH}/ukbb_output/ecg_rest_pretraining_mask_transfer/ecg_rest_pretraining_mask_transfer.hd5',
            '--u_connect', 
            '--inspect_model',
            '--training_steps', '15',
            '--validation_steps', '5',
            '--test_steps', '5',
            '--batch_size', '8',
            '--epochs', '400',
            '--patience', '40'            
           ]
args = parse_args()
model = make_multimodal_multitask_model(**args.__dict__)
for layer in model.layers:
    print(layer.trainable)

In [ ]:
args.tensor_maps_in[0].name = 'masked_strip'
args.tensor_maps_in[0].group = 'ecg_rest'

In [ ]:
run(args)